<a href="https://colab.research.google.com/github/fx-manmeet/learn-ml-repo/blob/main/Makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Solution to bigram model

It takes context into consideration

Eg,

The cat is walking in the bedroom



* A dog was running in a room

* The cat is running in a room

* A dog is walking in a bedroom

* The dog was walking in the room

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [ ]:
# download the names.txt file from github
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2024-06-10 09:16:56--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.05s   

2024-06-10 09:16:56 (4.34 MB/s) - ‘names.txt’ saved [228145/228145]



In [ ]:
words = open('names.txt', 'r').read().splitlines()
words[:8]


['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [ ]:
len(words)

32033

In [ ]:
#vocab of the character, and to/from conversion
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [ ]:
#build dataset

block_size = 3  #context length

X, Y = [], []
for w in words[:5]:

  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [ ]:
C = torch.randn((27,2))

In [ ]:
C[5]

tensor([-1.6612,  1.2760])

In [ ]:
F.one_hot(torch.tensor(5),num_classes= 27).float() @ C   # will mask others and give 5th indexed vector

tensor([-1.6612,  1.2760])

In [ ]:
emb = C[X]

In [ ]:
emb.shape

torch.Size([32, 3, 2])

In [ ]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [ ]:
emb[:,1,:].shape

torch.Size([32, 2])

In [ ]:
#works but torch has effecient way.
torch.cat(torch.unbind(emb, dim=1),1)

tensor([[ 0.9228, -0.2634,  0.9228, -0.2634,  0.9228, -0.2634],
        [ 0.9228, -0.2634,  0.9228, -0.2634, -1.6612,  1.2760],
        [ 0.9228, -0.2634, -1.6612,  1.2760,  0.3906,  1.6046],
        [-1.6612,  1.2760,  0.3906,  1.6046,  0.3906,  1.6046],
        [ 0.3906,  1.6046,  0.3906,  1.6046,  1.7050,  1.4339],
        [ 0.9228, -0.2634,  0.9228, -0.2634,  0.9228, -0.2634],
        [ 0.9228, -0.2634,  0.9228, -0.2634, -0.3990, -0.0548],
        [ 0.9228, -0.2634, -0.3990, -0.0548, -0.3127,  1.0707],
        [-0.3990, -0.0548, -0.3127,  1.0707, -0.2107,  0.6307],
        [-0.3127,  1.0707, -0.2107,  0.6307,  0.8031,  1.1241],
        [-0.2107,  0.6307,  0.8031,  1.1241, -0.2107,  0.6307],
        [ 0.8031,  1.1241, -0.2107,  0.6307,  1.7050,  1.4339],
        [ 0.9228, -0.2634,  0.9228, -0.2634,  0.9228, -0.2634],
        [ 0.9228, -0.2634,  0.9228, -0.2634,  1.7050,  1.4339],
        [ 0.9228, -0.2634,  1.7050,  1.4339,  0.8031,  1.1241],
        [ 1.7050,  1.4339,  0.8031,  1.1

In [ ]:
a =torch.arange(20)

In [ ]:
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])

In [ ]:
a.view((2,5,2))

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5],
         [ 6,  7],
         [ 8,  9]],

        [[10, 11],
         [12, 13],
         [14, 15],
         [16, 17],
         [18, 19]]])

In [ ]:
# every tensor in torch is stored in single dimension for computation
a.storage()

<ipython-input-19-24ecd8ebaa08>:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 20]

In [ ]:
#efficient
emb.view((32,6))

tensor([[ 0.9228, -0.2634,  0.9228, -0.2634,  0.9228, -0.2634],
        [ 0.9228, -0.2634,  0.9228, -0.2634, -1.6612,  1.2760],
        [ 0.9228, -0.2634, -1.6612,  1.2760,  0.3906,  1.6046],
        [-1.6612,  1.2760,  0.3906,  1.6046,  0.3906,  1.6046],
        [ 0.3906,  1.6046,  0.3906,  1.6046,  1.7050,  1.4339],
        [ 0.9228, -0.2634,  0.9228, -0.2634,  0.9228, -0.2634],
        [ 0.9228, -0.2634,  0.9228, -0.2634, -0.3990, -0.0548],
        [ 0.9228, -0.2634, -0.3990, -0.0548, -0.3127,  1.0707],
        [-0.3990, -0.0548, -0.3127,  1.0707, -0.2107,  0.6307],
        [-0.3127,  1.0707, -0.2107,  0.6307,  0.8031,  1.1241],
        [-0.2107,  0.6307,  0.8031,  1.1241, -0.2107,  0.6307],
        [ 0.8031,  1.1241, -0.2107,  0.6307,  1.7050,  1.4339],
        [ 0.9228, -0.2634,  0.9228, -0.2634,  0.9228, -0.2634],
        [ 0.9228, -0.2634,  0.9228, -0.2634,  1.7050,  1.4339],
        [ 0.9228, -0.2634,  1.7050,  1.4339,  0.8031,  1.1241],
        [ 1.7050,  1.4339,  0.8031,  1.1

In [ ]:
#so mul can work as
h= torch.tanh(emb.view(emb.shape[0],6) @ W1 +b1)     #hidden state with tanh nonlinearity
#h= emb.view(-1,6) @ W1 +b1               #works

In [ ]:
h.shape

torch.Size([32, 100])

In [ ]:
W2 = torch.randn(100,27)
b2 = torch.randn(27)

In [ ]:
logits= h @ W2 +b2

In [ ]:
logits.shape

torch.Size([32, 27])

In [ ]:
counts = logits.exp()

In [ ]:
prob = counts / counts.sum(1, keepdim= True)

In [ ]:
prob[1]

tensor([3.8013e-16, 9.7014e-01, 3.6550e-08, 1.0382e-13, 2.1760e-14, 3.4425e-12,
        5.5768e-06, 4.8400e-09, 3.5198e-14, 4.9310e-14, 1.9864e-05, 1.9878e-13,
        1.2208e-16, 1.0618e-08, 2.8835e-12, 2.9545e-02, 5.9023e-09, 1.6186e-16,
        7.2011e-13, 3.0234e-06, 9.9752e-09, 5.1508e-11, 1.1860e-11, 1.4439e-08,
        2.9027e-04, 8.0061e-10, 3.7515e-14])

In [ ]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [ ]:
loss= -prob[torch.arange(32),Y].log().mean()

In [ ]:
loss

tensor(16.4380)

In [ ]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters)

11897

In [ ]:
for p in parameters:
    p.requires_grad = True

In [ ]:
for i in range(10):

  # minibatch construct
  ix = torch.randint(0, X.shape[0], (32,))

  # forward pass
  emb = C[X[ix]] # (32, 3, 2)
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
  logits = h @ W2 + b2 # (32, 27)
  loss = F.cross_entropy(logits, Y[ix])
  #print(loss.item())

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()

  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

print(loss.item())

2.9426345825195312


In [ ]:
#revice it after a while
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

olidikdixjyrnuxjn.
olhinekkhgkbynrrhshrhrzothxjy.
olhinekkaalddaddrzvwucovwtesjrhguyozvwucovwthslrhcewaejjrhguyozvwucovwthslrhcgfccamhjziyoaeijtyofvwolhinekkaadzdyxzvkhhjkrhcvyhclyikkoovwtesjrhguyrrhshrhrzothxjy.
olhinekkaaxsyxshvwyhshrhrzovwthslrhcoyhshrhrzothxjy.
olsikdixjy.
olviuhjryhcoewikkogndrewaezjtyofvwolhinekkajdixjy.
oliikkogndrewaezjtyofwikyojvwucovwthslrhcgfccamzjkkhgyekkoovwtesjrhguyozvwucovwthslrhxvkccyhguyozvwucovwthslrhxvymhjbyhigfy.
olviuhjryhcaddrzvwucovwthslrhcgfjyrnuxjnhguyozvwucovwtesjrhguyozvwthslrhcadflhoewikyoaesza.
olhinekkaadzdyxzvkhhjkrhxvymhjziyoaeijt.
olxrhwaddrwuwdtesjrhguyozvwucovwixfkcovwtesjrhguyozvwucovwthslrhcgfccamhjziyoaeijt.
oliikkoovwthslrtnxwjkfinouhm.
olhinekkoovwtesjrhguyozvwucovwtesjrhguyozvwucovwtesjrhcgfccamhjziyoaeijt.
olidikdixjyrnnxvymhjziyoaeijrhguyozvwucovwthsjkhgkbynrrhwvkshxjy.
olxrngtesjrhgdiddaddrzvwucovwtesjrhgubdxshvwyhshrhrzothxjy.
oliikkogndzxvkcckkhgkbynrrhshrhrzothxjy.
olidikdixjy.
ols.
oliikkoovwtesjrhglbinekkoovwtesjrhguyozv